In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('../code')


from IPython.core.pylabtools import figsize
figsize(14, 7)
from pylab import *
rcParams['figure.subplot.wspace'] = 0.8
rcParams['figure.subplot.hspace'] = 0.6
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
rcParams['font.size'] = 16
rcParams['axes.titlesize'] = 16
rcParams['axes.labelsize'] = 16
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['legend.frameon'] = False


from hopfieldNetwork import hopfieldNet
from solverFile import solverClass
import numpy as np
import copy
from scipy.special import expit
import matplotlib.pyplot as plt



def to_triangular(matrix):
    return matrix[np.triu_indices_from(matrix, 1)]


def from_triangular(size, arr, diagonal_value):
    matrix = np.zeros([size, size])
    matrix[np.triu_indices_from(matrix, 1)] = arr
    matrix += matrix.T
    np.fill_diagonal(matrix, diagonal_value)
    return matrix


def dice_coefficient(p1, p2):
    p = 2 * np.sum(np.floor(0.6*(p1 + p2)))
    n = np.sum(p1)+np.sum(p2)
    return p/n


def evaluate_stability(network, patterns, average=True):
    error = np.empty(len(patterns))
    for i, p in enumerate(patterns):
        network.present_pattern(p)
        network.step(eval_epochs)
        output = netFisher.s
        error[i] = dice_coefficient(p, output)
    if average:
        return np.mean(error)
    return error

%matplotlib inline



In [77]:

ETA = 0.1  # learning rate
SPARSITY = 0.1  # number of zeros: SPARSITY = 0.1 means 10% ones and 90% zeros
IMAGE_SIZE = 10  # the size of our pattern will be (IMAGE_SIZE x IMAGE_SIZE)
print(IMAGE_SIZE**2*0.16)
eval_f = 1  # evaluation frequency (every eval_f-th iteration)
TRIALS = 1  # number of trials over which the results will be averaged
less_changed_weight_value = 0.00
# the learning rate of weights which are considered important have a
# learning rate of ETA * less_changed_weight_value
epochs_patterns_presented = 2
n_stored_patterns = 0
n_new_patterns = 20
n_tot_patterns = n_stored_patterns + n_new_patterns  # n of patterns created
NTRAIN = epochs_patterns_presented * n_new_patterns  # number of epochs
number_of_changed_values = 4600
number_of_changed_values = 4500
number_of_changed_values = int(IMAGE_SIZE*(IMAGE_SIZE-1)/2)-1200
number_of_changed_values = IMAGE_SIZE**2*(IMAGE_SIZE**2-1)//2//100*95
print(IMAGE_SIZE**2*(IMAGE_SIZE**2-1)//2,number_of_changed_values)
# number_of_changed_values_diag_0 = number_of_changed_values + IMAGE_SIZE**2/2
# the number of weigths that are changed is 2*number_of_changed_values
# (The factor of 2 is because of the symmetry of the weight matrix)
eval_epochs = 10  # how many steps are run before dice coefficient computed

# preparing patterns
solver = solverClass()
patterns = solver.create_patterns(SPARSITY, IMAGE_SIZE, n_tot_patterns)
netFisher = hopfieldNet(IMAGE_SIZE, ETA, SPARSITY)
original_patterns = copy.deepcopy(patterns)
patterns = patterns - SPARSITY

# learning patterns
p = np.random.random((IMAGE_SIZE**2, IMAGE_SIZE**2))
p = np.zeros(shape=(IMAGE_SIZE**2, IMAGE_SIZE**2))    

if n_stored_patterns>0:
    for i in range(n_stored_patterns):
        p += np.outer(patterns[:, i], patterns[:, i])
        netFisher.append_pattern(patterns[:, i], NTRAIN)
    w1 = p/n_stored_patterns
    ############################## RUN PRE-TEST ##############################
    netFisher.set_weights(w1)
    overall_error = 0
    for i in range(int(n_stored_patterns)):
        netFisher.present_pattern(original_patterns[:,i])
        netFisher.step(100)
        output = netFisher.s
        error = np.sum(original_patterns[:,i]-output)**2
        overall_error += error
    print('The overall_error is:   ', overall_error) # Error should be 0
else:
    ns = 20
    pre_patterns = solver.create_patterns(SPARSITY, IMAGE_SIZE, ns) - SPARSITY
    for i in range(ns):
        p += np.outer(pre_patterns[:, i], pre_patterns[:, i])
    w1 = p/ns/1
netFisher.set_weights(w1)



16.0
4950 4655


In [78]:
def compute_dice(c, ETA):#, t = 0.0):
    wF = np.copy(w1)
    DICE = -np.ones(shape=(NTRAIN, n_tot_patterns))
    N_PRE = 0
    NTRAIN_PRE = epochs_patterns_presented * N_PRE  # number of epochs
    pre_patterns = solver.create_patterns(SPARSITY, IMAGE_SIZE, N_PRE)
    all_patterns = np.hstack((pre_patterns - SPARSITY,patterns))
    w_mean = np.zeros(NTRAIN+NTRAIN_PRE)
    for epoch in range(NTRAIN+NTRAIN_PRE):
        id_pattern_taught = n_stored_patterns+epoch//epochs_patterns_presented
        pattern_taught = all_patterns[:, id_pattern_taught]

        z = (np.outer(pattern_taught, pattern_taught) - wF)

#         netFisher.curvature = np.abs(wF*(1-wF)-4*wF)  # not an actual curvature
        netFisher.curvature = np.abs(wF)  # not an actual curvature
        weight_perturbation = c/(c+netFisher.curvature)#np.exp(-netFisher.curvature*41)#fim_scale)

        perturbation_vector = weight_perturbation * z * ETA #* (np.abs(z)>t)
        wF = wF + perturbation_vector
        netFisher.set_weights(wF)
        w_mean[epoch] = np.mean(wF)

        # checking stability of patterns after eval_epochs iterations
        if epoch>=NTRAIN_PRE:
            DICE[epoch-NTRAIN_PRE] = evaluate_stability(  # old patterns
                netFisher, original_patterns.T[:n_tot_patterns], average=False)
    return DICE

def compute_dice_error(par):
#     c,t = par
    c, ETA = par
    return -np.sum(compute_dice(c,ETA)>0.9)


In [79]:
from scipy.optimize import minimize
from scipy import optimize

c = optimize.fmin(compute_dice_error,(0.1,0.1),disp=True)
print(c)


Optimization terminated successfully.
         Current function value: -335.000000
         Iterations: 46
         Function evaluations: 107
[0.0067162  0.74382889]


In [1]:
data = compute_dice(c[0],c[1])
fig = plt.figure(figsize=(15, 3))
ax = plt.subplot(111)
center = data#np.mean(data, axis=1)
print(np.mean(center))
n,m = center.shape
im = ax.imshow(center.T,cmap=plt.cm.summer,aspect=0.2*n/m,vmin=0,vmax=1)


NameError: name 'compute_dice' is not defined